In [240]:
#Problem 1

import numpy as np
from scipy.sparse import dok_matrix
import math
from numpy import linalg as la

def getad (file, N):
    
    try:
        data = dok_matrix((N,N), int)
        
        with open(file, "r") as infile:
            for line in infile:
                try: 
                    inn, out = line.strip().split()
                    data[int(inn), int(out)] = 1
                except:
                    pass
            
    except FileNotFoundError:
        print ("File not found")
    
    return data
    
admax = getad("/Users/bryanchia/Desktop/BootCamp2017/ProbSets/Comp/Wk4_DifIntOpt/matrix.txt", 8)
print(admax)

A = admax.toarray()
print(A)

  (0, 7)	1.0
  (1, 0)	1.0
  (3, 0)	1.0
  (3, 2)	1.0
  (3, 6)	1.0
  (4, 0)	1.0
  (4, 5)	1.0
  (4, 6)	1.0
  (5, 0)	1.0
  (5, 6)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
[[ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]]


In [191]:
#Problem 2

def computeK (A):
    
    Am = np.copy(A)
    
    for i in range (0, np.shape(A)[0]):
        if np.all(Am[i] == 0.0):
            Am[i,:] = 1.0
            
    D = np.sum(Am, axis = 1)
    
    K = (la.inv(np.diag(D)) @ Am).T
    
    return K, Am

K, Am = computeK(A)
print(K)

[[ 0.          1.          0.125       0.33333333  0.33333333  0.5         1.
   1.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.33333333  0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.33333333  0.          0.
   0.        ]
 [ 0.          0.          0.125       0.33333333  0.33333333  0.5         0.
   0.        ]
 [ 1.          0.          0.125       0.          0.          0.          0.
   0.        ]]


In [255]:
#Problem 3

def solvep(Am, N = None, d = 0.85, tol = 1e-5):
    
    if N != None:
        Adj = Am[:N,:N]
    else:
        Adj = np.copy(Am)
    
    n = np.shape(Adj)[0]
    p = np.random.rand(n)
    p1 = np.zeros(n)
    K, Am = computeK(Adj)
    diff = 300
    
    while diff > tol:
        p = p1
        p1 = ((d * K) @ p) + ((1 - d) / n * np.ones(n))
        diff = la.norm(p1 - p)
    
    return p

p = solvep(Am)
print(p)

[ 0.43865334  0.02171029  0.02786154  0.02171029  0.02171029  0.02786154
  0.04585394  0.39455819]


In [256]:
#Problem 4

def near(a, b, rtol = 1e-5, atol = 1e-8):
    return np.abs(a-b)<(atol+rtol*np.abs(b))

def eigensolver(Am, N = None, d = 0.85):

    if N != None:
        Adj = Am[:N,:N]
    else:
        Adj = np.copy(Am)
    
    n = np.shape(Adj)[0]
    K, Am = computeK(Adj)
    eig_p = np.zeros(n)
    
    B = ((d * K) + ((1 - d) / n * np.ones((n,n))))
    
    eigval, eigvec = la.eig(B)
    
    eig_p = eigvec[near(eigval, 1.0)]
    eig_p = eig_p / np.sum(eig_p)
    return eig_p 

print(eigensolver(Am))

[[  5.51441294e-01   5.28265141e-01  -1.76697500e-01   7.06621715e-17
    9.69910648e-02   2.07568214e-09  -2.07568185e-09   1.39091157e-16]]


In [307]:
#Problem 5

import pandas as pd

def convert(filename):
    
        teams_pd = pd.read_csv(filename, delimiter =",", header = 0)
        teams = (teams_pd).as_matrix(columns=None)
        
        nodes = np.zeros_like(teams)
        teamlist = np.array(([teams[0,0]], [teams[0,1]]))
        team_check = ''
        
        #Give each name an index

        for i in range (0, np.shape(teams)[0]):
            
            for j in range (0, np.shape(teams)[1]):
                
                team_check = teams[i,j]
                
                if np.all(teamlist != team_check):
                    teamlist = np.append(teamlist, team_check)
        
        index = np.arange(0, len(teamlist))
        
        #Replace each name by an index number 
        
        m, n = np.shape(teams)
        
        team_indices = np.zeros_like(teams)
        
        for i in range (0, np.shape(teams)[0]):
            
            for j in range (0, np.shape(teams)[1]):
                
                team_check = teams[i,j]
                
                for k in range (0, len(teamlist)):
                
                    if team_check == teamlist[k]:
                        
                        team_indices[i,j] = index[k]
        
        #Create adjacency matrix
        
        admat = np.zeros((len(teamlist), len(teamlist)))
        
        for a in range (0, np.shape(team_indices)[0]):
            
            i = team_indices[a, 1]
            j = team_indices[a, 0]
            
            admat[i,j] += 1
        
        return admat, teamlist

admat, teamlist = convert("/Users/bryanchia/Desktop/BootCamp2017/ProbSets/Comp/Wk4_DifIntOpt/ncaa2013.csv")

pncaa = - solvep(admat, d = 0.7)
rankings = teamlist[np.argsort(pncaa)]
print(rankings)

['Duke' 'Kansas' 'Indiana' 'Louisville' 'St Louis' 'Miami FL' 'Butler'
 'Wisconsin' 'Illinois' 'New Mexico' 'Ohio St' 'Michigan St' 'Georgetown'
 'Syracuse' 'Michigan' 'Minnesota' 'UCLA' 'UNLV' 'Gonzaga' 'Florida'
 'Marquette' 'VA Commonwealth' 'Arizona' 'Notre Dame' 'Oklahoma St'
 'Villanova' 'Creighton' 'Temple' 'Maryland' 'Virginia' 'NC State' 'Oregon'
 'Missouri' 'Xavier' 'Kansas St' 'Pittsburgh' 'Colorado' 'North Carolina'
 'San Diego St' 'Wichita St' 'Cincinnati' 'Oklahoma' 'California'
 'Mississippi' 'Connecticut' 'Indiana St' 'Memphis' 'Colorado St' 'Iowa St'
 'Tennessee' 'Kentucky' 'La Salle' 'Boise St' 'Baylor' 'Belmont'
 'Washington' 'Charlotte' "St Mary's CA" 'Texas A&M' 'Arkansas'
 'S Dakota St' 'Florida St' 'Massachusetts' 'Air Force' 'Iowa' 'Akron'
 'Alabama' 'Arizona St' 'Georgia Tech' 'Northern Iowa' 'Middle Tenn St'
 'Stanford' 'Wake Forest' 'Bucknell' "St Joseph's PA" 'Providence'
 'Evansville' 'USC' 'Valparaiso' 'FL Gulf Coast' "St John's" 'LSU' 'Utah'
 'Illinois St